# 🚀 NOTEBOOK 5 : DEEP LEARNING OPTIMISÉ (GPU 4GB)

## Objectifs
- ✅ Atteindre **85%+ accuracy**
- ✅ **< 30 minutes** d'entraînement par modèle
- ✅ Optimisé pour **GPU 4GB** + **~150 images/classe**
- ✅ Transfer Learning (EfficientNetV2-S)
- ✅ Mixed Precision Training (FP16)
- ✅ Sauvegarder modèle pour API

## Stratégie
1. Transfer Learning depuis ImageNet
2. Progressive unfreezing (3 phases)
3. Data augmentation intensive
4. MixUp régularisation
5. Mixed precision (économie mémoire)

**Basé sur la recherche 2024-2025 : EfficientNetV2 + PyTorch pour meilleur compromis vitesse/précision**

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import torchvision
from torchvision import transforms, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import json
import time

# Vérifier GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎮 Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 1. CONFIGURATION

In [ ]:
# Chemins
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "Data"
IMAGES_DIR = DATA_DIR / "Images"
MODELS_DIR = BASE_DIR / "models"
RESULTS_DIR = BASE_DIR / "results" / "deep_learning"
MODELS_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

METADATA_FILE = DATA_DIR / "data_images_corrected.csv"

# Hyperparamètres (optimisés pour GPU 4GB)
IMG_SIZE = 224
BATCH_SIZE = 16  # Safe pour 4GB avec FP16
EPOCHS_PHASE1 = 3  # Classification head seulement
EPOCHS_PHASE2 = 5  # Partial unfreeze
EPOCHS_PHASE3 = 7  # Full fine-tune
TOTAL_EPOCHS = EPOCHS_PHASE1 + EPOCHS_PHASE2 + EPOCHS_PHASE3
NUM_WORKERS = 4
RANDOM_STATE = 42

CATEGORIES = [
    "Baby Care",
    "Beauty and Personal Care",
    "Computers",
    "Home Decor & Festive Needs",
    "Home Furnishing",
    "Kitchen & Dining",
    "Watches"
]
NUM_CLASSES = len(CATEGORIES)

print(f"📦 Config:")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Epochs: {TOTAL_EPOCHS} (3+5+7 phases)")
print(f"   Classes: {NUM_CLASSES}")
print(f"   Image size: {IMG_SIZE}×{IMG_SIZE}")

## 2. CHARGEMENT DONNÉES

In [ ]:
# Charger métadonnées
df = pd.read_csv(METADATA_FILE)
print(f"Dataset: {len(df)} images")

# Vérifier images
valid_idx = [i for i, row in df.iterrows() if (IMAGES_DIR / row['image']).exists()]
df = df.loc[valid_idx].reset_index(drop=True)
print(f"✅ {len(df)} images valides")

# Encoder labels
label_map = {cat: i for i, cat in enumerate(CATEGORIES)}
df['label'] = df['main_category'].map(label_map)

# Split 70/15/15
train_df, temp_df = train_test_split(
    df, test_size=0.3, random_state=RANDOM_STATE, stratify=df['label']
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=RANDOM_STATE, stratify=temp_df['label']
)

print(f"\n📊 Split:")
print(f"   Train: {len(train_df)}")
print(f"   Val:   {len(val_df)}")
print(f"   Test:  {len(test_df)}")

## 3. DATA AUGMENTATION

**Basé sur recherche 2024**: TrivialAugment pour simplicité + performances

In [ ]:
# Normalisation ImageNet (requis pour transfer learning)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# Augmentation train (intensive)
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.75, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.TrivialAugmentWide(),  # Auto-augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# Val/Test (sans augmentation)
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

print("✅ Transforms configurés")

## 4. DATASET CUSTOM

In [ ]:
class ProductDataset(Dataset):
    """Dataset custom pour produits"""
    
    def __init__(self, dataframe, images_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.images_dir = Path(images_dir)
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.images_dir / row['image']
        
        # Charger image
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label = row['label']
        
        return image, label

# Créer datasets
train_dataset = ProductDataset(train_df, IMAGES_DIR, train_transform)
val_dataset = ProductDataset(val_df, IMAGES_DIR, val_transform)
test_dataset = ProductDataset(test_df, IMAGES_DIR, val_transform)

# Dataloaders
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)

print(f"✅ Dataloaders créés")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches:   {len(val_loader)}")
print(f"   Test batches:  {len(test_loader)}")

## 5. MODÈLE : EfficientNetV2-S

**Choix basé sur recherche**: Meilleur compromis vitesse/précision pour GPU 4GB

In [ ]:
def build_model(num_classes=7, pretrained=True):
    """EfficientNetV2-S avec classification head custom"""
    
    # Base pré-entraînée
    model = models.efficientnet_v2_s(weights='IMAGENET1K_V1' if pretrained else None)
    
    # Remplacer classifier
    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.4),
        nn.Linear(in_features, 512),
        nn.ReLU(),
        nn.BatchNorm1d(512),
        nn.Dropout(p=0.3),
        nn.Linear(512, num_classes)
    )
    
    return model

# Créer modèle
model = build_model(num_classes=NUM_CLASSES)
model = model.to(device)

print("✅ Modèle créé")
print(f"   Total params: {sum(p.numel() for p in model.parameters()):,}")
print(f"   Trainable: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

## 6. MIXUP RÉGULARISATION

In [ ]:
def mixup_data(x, y, alpha=0.2):
    """MixUp: mélanger images et labels"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(device)
    
    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """Loss pour MixUp"""
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

print("✅ MixUp configuré (alpha=0.2)")

## 7. FONCTIONS D'ENTRAÎNEMENT

In [ ]:
def train_epoch(model, loader, criterion, optimizer, scaler, use_mixup=True):
    """Entraîner 1 epoch avec mixed precision"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc="Train")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        # MixUp
        if use_mixup:
            images, labels_a, labels_b, lam = mixup_data(images, labels, alpha=0.2)
        
        optimizer.zero_grad()
        
        # Mixed precision
        with autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(images)
            
            if use_mixup:
                loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
            else:
                loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * images.size(0)
        
        # Accuracy (sans MixUp pour simplicité)
        _, predicted = outputs.max(1)
        total += labels.size(0) if not use_mixup else labels_a.size(0)
        if use_mixup:
            correct += (lam * predicted.eq(labels_a).sum().item() + 
                       (1-lam) * predicted.eq(labels_b).sum().item())
        else:
            correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({'loss': loss.item(), 'acc': 100.*correct/total})
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

def val_epoch(model, loader, criterion):
    """Validation epoch"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(loader, desc="Val")
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            pbar.set_postfix({'loss': loss.item(), 'acc': 100.*correct/total})
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

print("✅ Fonctions train/val prêtes")

## 8. ENTRAÎNEMENT PROGRESSIF (3 PHASES)

### PHASE 1 : Classification Head Seulement (3 epochs)

In [ ]:
print("="*70)
print("PHASE 1 : CLASSIFICATION HEAD (backbone gelé)")
print("="*70)

# Geler backbone
for param in model.features.parameters():
    param.requires_grad = False

# Optimizer
optimizer = optim.AdamW(model.classifier.parameters(), lr=1e-3, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scaler = GradScaler()

# Training
history_p1 = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_acc_p1 = 0.0

start_time = time.time()

for epoch in range(EPOCHS_PHASE1):
    print(f"\nEpoch {epoch+1}/{EPOCHS_PHASE1}")
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scaler, use_mixup=True)
    val_loss, val_acc = val_epoch(model, val_loader, criterion)
    
    history_p1['train_loss'].append(train_loss)
    history_p1['train_acc'].append(train_acc)
    history_p1['val_loss'].append(val_loss)
    history_p1['val_acc'].append(val_acc)
    
    print(f"Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
    print(f"Val:   Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    
    if val_acc > best_acc_p1:
        best_acc_p1 = val_acc
        torch.save(model.state_dict(), MODELS_DIR / 'best_phase1.pth')
        print(f"✅ Meilleur modèle Phase 1: {best_acc_p1:.2f}%")

phase1_time = time.time() - start_time
print(f"\n⏱️ Phase 1: {phase1_time/60:.1f} min")
print(f"✅ Best Val Acc Phase 1: {best_acc_p1:.2f}%")

### PHASE 2 : Partial Unfreeze (5 epochs)

In [ ]:
print("\n" + "="*70)
print("PHASE 2 : PARTIAL UNFREEZE (derniers blocks)")
print("="*70)

# Charger meilleur modèle Phase 1
model.load_state_dict(torch.load(MODELS_DIR / 'best_phase1.pth'))

# Dégeler derniers 30% du backbone
total_layers = len(list(model.features.parameters()))
unfreeze_from = int(total_layers * 0.7)

for i, param in enumerate(model.features.parameters()):
    if i >= unfreeze_from:
        param.requires_grad = True

print(f"Dégel de {total_layers - unfreeze_from}/{total_layers} couches")

# Optimizer avec learning rates différenciés
optimizer = optim.AdamW([
    {'params': model.features.parameters(), 'lr': 1e-4},
    {'params': model.classifier.parameters(), 'lr': 1e-3}
], weight_decay=1e-4)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS_PHASE2)

# Training
history_p2 = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_acc_p2 = 0.0

start_time = time.time()

for epoch in range(EPOCHS_PHASE2):
    print(f"\nEpoch {epoch+1}/{EPOCHS_PHASE2}")
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scaler)
    val_loss, val_acc = val_epoch(model, val_loader, criterion)
    
    history_p2['train_loss'].append(train_loss)
    history_p2['train_acc'].append(train_acc)
    history_p2['val_loss'].append(val_loss)
    history_p2['val_acc'].append(val_acc)
    
    print(f"Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
    print(f"Val:   Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    
    if val_acc > best_acc_p2:
        best_acc_p2 = val_acc
        torch.save(model.state_dict(), MODELS_DIR / 'best_phase2.pth')
        print(f"✅ Meilleur modèle Phase 2: {best_acc_p2:.2f}%")
    
    scheduler.step()

phase2_time = time.time() - start_time
print(f"\n⏱️ Phase 2: {phase2_time/60:.1f} min")
print(f"✅ Best Val Acc Phase 2: {best_acc_p2:.2f}%")

### PHASE 3 : Full Fine-Tuning (7 epochs)

In [ ]:
print("\n" + "="*70)
print("PHASE 3 : FULL FINE-TUNING")
print("="*70)

# Charger meilleur modèle Phase 2
model.load_state_dict(torch.load(MODELS_DIR / 'best_phase2.pth'))

# Dégeler tout
for param in model.parameters():
    param.requires_grad = True

# Optimizer avec LR très faible
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS_PHASE3)

# Training
history_p3 = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_acc_p3 = 0.0

start_time = time.time()

for epoch in range(EPOCHS_PHASE3):
    print(f"\nEpoch {epoch+1}/{EPOCHS_PHASE3}")
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scaler)
    val_loss, val_acc = val_epoch(model, val_loader, criterion)
    
    history_p3['train_loss'].append(train_loss)
    history_p3['train_acc'].append(train_acc)
    history_p3['val_loss'].append(val_loss)
    history_p3['val_acc'].append(val_acc)
    
    print(f"Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
    print(f"Val:   Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    
    if val_acc > best_acc_p3:
        best_acc_p3 = val_acc
        torch.save(model.state_dict(), MODELS_DIR / 'best_final.pth')
        print(f"✅ Meilleur modèle Final: {best_acc_p3:.2f}%")
    
    scheduler.step()

phase3_time = time.time() - start_time
print(f"\n⏱️ Phase 3: {phase3_time/60:.1f} min")
print(f"✅ Best Val Acc Final: {best_acc_p3:.2f}%")

total_time = phase1_time + phase2_time + phase3_time
print(f"\n⏱️ TEMPS TOTAL: {total_time/60:.1f} minutes")

## 9. ÉVALUATION SUR TEST SET

In [ ]:
print("="*70)
print("ÉVALUATION FINALE")
print("="*70)

# Charger meilleur modèle
model.load_state_dict(torch.load(MODELS_DIR / 'best_final.pth'))
model.eval()

# Prédictions
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Test"):
        images = images.to(device)
        outputs = model(images)
        _, preds = outputs.max(1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Accuracy
test_acc = accuracy_score(all_labels, all_preds)

print(f"\n📊 TEST ACCURACY: {test_acc*100:.2f}%")

# Classification report
print("\n" + "="*70)
print("RAPPORT PAR CATÉGORIE")
print("="*70)
print(classification_report(all_labels, all_preds, target_names=CATEGORIES))

### Matrice de Confusion

In [ ]:
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=CATEGORIES, yticklabels=CATEGORIES,
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted', fontweight='bold')
plt.ylabel('Actual', fontweight='bold')
plt.title(f'Confusion Matrix - Test Accuracy: {test_acc*100:.2f}%', 
          fontweight='bold', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

### Courbes d'apprentissage

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Combiner historiques
all_train_loss = history_p1['train_loss'] + history_p2['train_loss'] + history_p3['train_loss']
all_val_loss = history_p1['val_loss'] + history_p2['val_loss'] + history_p3['val_loss']
all_train_acc = history_p1['train_acc'] + history_p2['train_acc'] + history_p3['train_acc']
all_val_acc = history_p1['val_acc'] + history_p2['val_acc'] + history_p3['val_acc']

epochs = range(1, len(all_train_loss) + 1)

# Loss
ax1.plot(epochs, all_train_loss, 'b-', label='Train', linewidth=2)
ax1.plot(epochs, all_val_loss, 'r-', label='Val', linewidth=2)
ax1.axvline(x=EPOCHS_PHASE1, color='gray', linestyle='--', alpha=0.5, label='Phase 1→2')
ax1.axvline(x=EPOCHS_PHASE1+EPOCHS_PHASE2, color='gray', linestyle='--', alpha=0.5, label='Phase 2→3')
ax1.set_xlabel('Epoch', fontweight='bold')
ax1.set_ylabel('Loss', fontweight='bold')
ax1.set_title('Loss Curves', fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Accuracy
ax2.plot(epochs, all_train_acc, 'b-', label='Train', linewidth=2)
ax2.plot(epochs, all_val_acc, 'r-', label='Val', linewidth=2)
ax2.axvline(x=EPOCHS_PHASE1, color='gray', linestyle='--', alpha=0.5)
ax2.axvline(x=EPOCHS_PHASE1+EPOCHS_PHASE2, color='gray', linestyle='--', alpha=0.5)
ax2.axhline(y=85, color='green', linestyle='--', alpha=0.5, label='Objectif 85%')
ax2.set_xlabel('Epoch', fontweight='bold')
ax2.set_ylabel('Accuracy (%)', fontweight='bold')
ax2.set_title('Accuracy Curves', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

## 10. SAUVEGARDER POUR API

In [ ]:
# Sauvegarder modèle final pour API
final_model_path = MODELS_DIR / 'cnn_final.keras'

# PyTorch → ONNX (meilleur pour déploiement)
model.eval()
dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)

torch.onnx.export(
    model, dummy_input,
    MODELS_DIR / 'cnn_final.onnx',
    input_names=['image'],
    output_names=['predictions'],
    dynamic_axes={'image': {0: 'batch'}, 'predictions': {0: 'batch'}},
    opset_version=17
)

print("✅ Modèle ONNX exporté")

# Sauvegarder aussi en .pth
torch.save(model.state_dict(), MODELS_DIR / 'cnn_final.pth')

# Label encoder
joblib.dump(label_map, MODELS_DIR / 'label_enconders.pkl')

print("✅ Label encoder sauvegardé")

### Sauvegarder métriques

In [ ]:
metrics = {
    'model': 'EfficientNetV2-S',
    'test_accuracy': float(test_acc),
    'best_val_acc_phase1': float(best_acc_p1),
    'best_val_acc_phase2': float(best_acc_p2),
    'best_val_acc_final': float(best_acc_p3),
    'total_epochs': TOTAL_EPOCHS,
    'training_time_minutes': float(total_time / 60),
    'batch_size': BATCH_SIZE,
    'img_size': IMG_SIZE,
    'num_classes': NUM_CLASSES,
    'categories': CATEGORIES
}

with open(RESULTS_DIR / 'metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\n" + "="*70)
print("✅ ENTRAÎNEMENT TERMINÉ")
print("="*70)
print(f"\n📊 Résultats finaux:")
print(f"   Test Accuracy: {test_acc*100:.2f}%")
print(f"   Temps total: {total_time/60:.1f} minutes")
print(f"\n📁 Fichiers sauvegardés:")
print(f"   - Modèle: {MODELS_DIR}/cnn_final.onnx")
print(f"   - Modèle: {MODELS_DIR}/cnn_final.pth")
print(f"   - Encoder: {MODELS_DIR}/label_enconders.pkl")
print(f"   - Métriques: {RESULTS_DIR}/metrics.json")

if test_acc >= 0.85:
    print("\n🎉 OBJECTIF ATTEINT : Accuracy ≥ 85% !")
else:
    print(f"\n⚠️ Accuracy sous objectif ({test_acc*100:.2f}% < 85%)")
    print("   Recommandations:")
    print("   - Augmenter epochs Phase 3")
    print("   - Ajouter plus d'augmentation")
    print("   - Essayer EfficientNetB0 (plus petit)")